# <font color=red> Importing the data and creating data frame <font color/>

### importing the table from wikipedia

In [1]:
import pandas as pd

In [2]:
##using request to get the HTML of wikipedia page and beautifulsoup to parse it

import requests
wiki_html=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

    
from bs4 import BeautifulSoup
soup = BeautifulSoup(wiki_html,'lxml')
#print(soup.prettify())


In [3]:
#creating the dataframe. Assumed that we already knew the size of the table

toronto_table_html = soup.find_all('table')[0]
#print (toronto_table)

toronto = pd.DataFrame(columns=['Postcode','Borough','Neighborhood'], index = range(0,288))
    
row_marker = 0

for row in toronto_table_html.find_all('tr'):
        column_marker = 0
        columns = row.find_all('td')
        for column in columns:
            toronto.iat[row_marker,column_marker] = column.get_text()
            column_marker= column_marker + 1
        if len(columns) > 0:
                    row_marker += 1


In [4]:
##cleaning the data

toronto = toronto.replace('\n',' ', regex=True)

toronto1= toronto[toronto.Borough!='Not assigned']

toronto2=toronto1.groupby(['Postcode','Borough'], as_index=False).agg(lambda x: ','.join(x))


In [5]:
na_neigh_rows = toronto2.Neighborhood == 'Not assigned'
toronto2.loc[na_neigh_rows, 'Neighborhood'] = toronto2.loc[na_neigh_rows, 'Borough']
toronto2[1:5]

,Postcode,Borough,Neighborhood
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union"
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
toronto2.shape

(103, 3)

# <font color=red> Adding the Geocordinates <font color/>

In [7]:
# The code was removed by Watson Studio for sharing.

In [8]:
geo_data=df_data_1

In [9]:
toronto_index= toronto2.set_index('Postcode')
geo_index = geo_data.set_index('Postal Code')
toronto_final = pd.concat([toronto_index, geo_index], axis=1, join='inner')

In [10]:
tf=toronto_final[toronto_final.Borough.str.contains('Toronto')]
tf[1:5]

,Borough,Neighborhood,Latitude,Longitude
M4K,East Toronto,"The Danforth West ,Riverdale",43.679557,-79.352188
M4L,East Toronto,"The Beaches West ,India Bazaar",43.668999,-79.315572
M4M,East Toronto,Studio District,43.659526,-79.340923
M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


# <font color=red> Analyzing the neighborhoods <font color/>

In [11]:
from geopy.geocoders import Nominatim

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#for Kmeans model
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

In [12]:
## latitude / longitude of toronto

address = 'Toronto, Toronto'

geolocator = Nominatim(user_agent="tt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [13]:
#create the map

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(tf['Latitude'], tf['Longitude'], tf['Borough'], tf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [14]:
CLIENT_ID = '01LMYPD3MFXSNZ1IUBC1JSBGWIETJNZ24EHJWYXJ0UYTAYOI' # your Foursquare ID
CLIENT_SECRET = 'RB5UPSPWTI4Q4GU0J2CWJVI5DKFAB1KHJACS4SYWR3MDYLCO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT=100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 01LMYPD3MFXSNZ1IUBC1JSBGWIETJNZ24EHJWYXJ0UYTAYOI
CLIENT_SECRET:RB5UPSPWTI4Q4GU0J2CWJVI5DKFAB1KHJACS4SYWR3MDYLCO


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
tor_venues = getNearbyVenues(names=tf['Neighborhood'],
                                   latitudes=tf['Latitude'],
                                   longitudes=tf['Longitude']
                                  )


The Beaches 
The Danforth West ,Riverdale 
The Beaches West ,India Bazaar 
Studio District 
Lawrence Park 
Davisville North 
North Toronto West 
Davisville 
Moore Park ,Summerhill East 
Deer Park ,Forest Hill SE ,Rathnelly ,South Hill ,Summerhill West 
Rosedale 
Cabbagetown ,St. James Town 
Church and Wellesley 
Harbourfront ,Regent Park 
Ryerson ,Garden District 
St. James Town 
Berczy Park 
Central Bay Street 
Adelaide ,King ,Richmond 
Harbourfront East ,Toronto Islands ,Union Station 
Design Exchange ,Toronto Dominion Centre 
Commerce Court ,Victoria Hotel 
Roselawn 
Forest Hill North ,Forest Hill West 
The Annex ,North Midtown ,Yorkville 
Harbord ,University of Toronto 
Chinatown ,Grange Park ,Kensington Market 
CN Tower ,Bathurst Quay ,Island airport ,Harbourfront West ,King and Spadina ,Railway Lands ,South Niagara 
Stn A PO Boxes 25 The Esplanade 
First Canadian Place ,Underground city 
Christie 
Dovercourt Village ,Dufferin 
Little Portugal ,Trinity 
Brockton ,Exhibition Place 

In [19]:
tor_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide ,King ,Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton ,Exhibition Place ,Parkdale Village",21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower ,Bathurst Quay ,Island airport ,Harbourfront West ,King and Spadina ,Railway Lands ,South Niagara",14,14,14,14,14,14
"Cabbagetown ,St. James Town",45,45,45,45,45,45
Central Bay Street,84,84,84,84,84,84
"Chinatown ,Grange Park ,Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15


In [20]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 239 uniques categories.


In [21]:
#tor_venues

In [22]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe


In [23]:

tor_onehot=tor_onehot.drop(columns=["Neighborhood"])


In [24]:
tor_onehot.insert(0,'Neighborhood',tor_venues['Neighborhood'],allow_duplicates = False) 

tor_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
tor_onehot[['Neighborhood']]

,Neighborhood
0,The Beaches
1,The Beaches
2,The Beaches
3,The Beaches
4,The Beaches
5,"The Danforth West ,Riverdale"
6,"The Danforth West ,Riverdale"
7,"The Danforth West ,Riverdale"
8,"The Danforth West ,Riverdale"
9,"The Danforth West ,Riverdale"


In [26]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
import numpy as np

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide ,King ,Richmond",Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse,Breakfast Spot,Gym,Asian Restaurant,Hotel,American Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Seafood Restaurant,Beer Bar,Café,Cheese Shop,Farmers Market,Bakery,Concert Hall
2,"Brockton ,Exhibition Place ,Parkdale Village",Breakfast Spot,Coffee Shop,Café,Italian Restaurant,Pet Store,Climbing Gym,Restaurant,Caribbean Restaurant,Burrito Place,Stadium
3,Business Reply Mail Processing Centre 969 East...,Light Rail Station,Yoga Studio,Garden Center,Farmers Market,Smoke Shop,Fast Food Restaurant,Brewery,Park,Burrito Place,Spa
4,"CN Tower ,Bathurst Quay ,Island airport ,Harbo...",Airport Lounge,Airport Service,Airport Terminal,Plane,Airport,Airport Food Court,Airport Gate,Coffee Shop,Harbor / Marina,Sculpture Garden


In [31]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:40] 

array([0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       4, 0, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 6, 0, 0], dtype=int32)

In [32]:

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = tf

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
M4E,East Toronto,The Beaches,43.676357,-79.293031,6,Trail,Pub,Health Food Store,Coffee Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Yoga Studio
M4K,East Toronto,"The Danforth West ,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Bookstore,Brewery,Bubble Tea Shop,Burger Joint
M4L,East Toronto,"The Beaches West ,India Bazaar",43.668999,-79.315572,0,Pizza Place,Sushi Restaurant,Ice Cream Shop,Italian Restaurant,Liquor Store,Fish & Chips Shop,Fast Food Restaurant,Movie Theater,Park,Pet Store
M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Convenience Store,Brewery,Seafood Restaurant,Sandwich Place
M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Bus Line,Dim Sum Restaurant,Swim School,Yoga Studio,Donut Shop,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market


In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters